**Important Modules**

In [ ]:
import keras
import tensorflow
from tensorflow.keras import Model
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import AvgPool2D, Concatenate, Flatten, Activation


**Xception-Network**

In [ ]:
def xception_network(input_shape,output_shape):
    def conv_bn(x, filters, kernel_size, strides=1):
        x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        return x
    
    def sep_bn(x, filters, kernel_size, strides=1):
        x = SeparableConv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        return x
    
    def entry_flow(x):
        x = conv_bn(x, filters=32, kernel_size=3, strides=2)
        x = ReLU()(x)
        x = conv_bn(x, filters=64, kernel_size=3)
        tensor = ReLU()(x)
    
        x = sep_bn(tensor, filters=128, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=128, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
        tensor = conv_bn(tensor, filters=128, kernel_size=1, strides=2)
    
        x = Add()([tensor, x])
        x = ReLU()(x)
        x = sep_bn(x, filters=256, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=256, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
        tensor = conv_bn(tensor, filters=256, kernel_size=1, strides=2)
    
        x = Add()([tensor, x])
        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
        tensor = conv_bn(tensor, filters=728, kernel_size=1, strides=2)
        x = Add()([tensor, x])
        return x
    
    def middle_flow(tensor):
        for _ in range(8):
            x = ReLU()(tensor)
            x = sep_bn(x, filters=728, kernel_size=3)
            x = ReLU()(x)
            x = sep_bn(x, filters=728, kernel_size=3)
            x = ReLU()(x)
            x = sep_bn(x, filters=728, kernel_size=3)
            tensor = Add()([tensor, x])
        return tensor

    def exit_flow(tensor):
        x = ReLU()(tensor)
        x = sep_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=1024, kernel_size=3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, filters=1024, kernel_size=1, strides=2)
        x = Add()([tensor, x])
        x = sep_bn(x, filters=1536, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=2048, kernel_size=3)
        x = ReLU()(x)
        x = GlobalAvgPool2D()(x)
        # Match it with number of output
        # Actual Activation function is softmax
        x = Dense(output_shape, activation='sigmoid')(x)
        return x

    input = Input(input_shape)
    x = entry_flow(input)
    x = middle_flow(x)
    output = exit_flow(x)
    model = Model(input, output)
    return model

xception_model = xception_network(shape=[224, 224, 3],1)

xception_model.summary()

**DenseNet-Network**

DenseNet-121

In [ ]:
def densenet_121_network(input_shape,output_shape):
    def bn_rl_conv(x, filters, kernel_size):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                padding='same')(x)
        return x

    def dense_block(tensor, k, reps):
        for _ in range(reps):
            x = bn_rl_conv(tensor, filters=4*k, kernel_size=1)
            x = bn_rl_conv(x, filters=k, kernel_size=3)
            tensor = Concatenate()([tensor, x])
        return tensor

    def transition_layer(x, theta):
        f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
        x = bn_rl_conv(x, filters=f, kernel_size=1)
        x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
        return x

    k = 32
    theta = 0.5
    repetitions = 6, 12, 24, 16
    
    input = Input(input_shape)
    
    x = Conv2D(2*k, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for reps in repetitions:
        d = dense_block(x, k, reps)
        x = transition_layer(d, theta)
    
    x = GlobalAvgPool2D()(d)
    # Actual Activation function is softmax
    output = Dense(output_shape, activation='softmax')(x)
    model = Model(input, output)

    return model


DenseNet-169

In [ ]:
def densenet_169_network(input_shape,output_shape):
    def bn_rl_conv(x, filters, kernel_size):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                padding='same')(x)
        return x

    def dense_block(tensor, k, reps):
        for _ in range(reps):
            x = bn_rl_conv(tensor, filters=4*k, kernel_size=1)
            x = bn_rl_conv(x, filters=k, kernel_size=3)
            tensor = Concatenate()([tensor, x])
        return tensor

    def transition_layer(x, theta):
        f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
        x = bn_rl_conv(x, filters=f, kernel_size=1)
        x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
        return x

    k = 32
    theta = 0.5
    repetitions = 6, 12, 32, 32
    
    input = Input(input_shape)
    
    x = Conv2D(2*k, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for reps in repetitions:
        d = dense_block(x, k, reps)
        x = transition_layer(d, theta)
    
    x = GlobalAvgPool2D()(d)
    # Actual Activation function is softmax
    output = Dense(output_shape, activation='softmax')(x)
    model = Model(input, output)

    return model


DenseNet-201

In [ ]:
def densenet_201_network(input_shape,output_shape):
    def bn_rl_conv(x, filters, kernel_size):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                padding='same')(x)
        return x

    def dense_block(tensor, k, reps):
        for _ in range(reps):
            x = bn_rl_conv(tensor, filters=4*k, kernel_size=1)
            x = bn_rl_conv(x, filters=k, kernel_size=3)
            tensor = Concatenate()([tensor, x])
        return tensor

    def transition_layer(x, theta):
        f = int(tensorflow.keras.backend.int_shape(x)[-1] * theta)
        x = bn_rl_conv(x, filters=f, kernel_size=1)
        x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
        return x

    k = 32
    theta = 0.5
    theta = 0.5
    repetitions = 6, 12, 48, 32
    
    input = Input(input_shape)
    
    x = Conv2D(2*k, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for reps in repetitions:
        d = dense_block(x, k, reps)
        x = transition_layer(d, theta)
    
    x = GlobalAvgPool2D()(d)
    # Actual Activation function is softmax
    output = Dense(output_shape, activation='softmax')(x)
    model = Model(input, output)

    return model

model = densenet_169_network([500,150,3],1)

**VGG-Network**

VGG-16

In [ ]:
# Creating VGG-16 Model

def vgg_16(input_shape,output_shape):
    model = Sequential()

    model.add(Conv2D(input_shape=input_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    # Actual activation function is softmax
    model.add(Dense(output_shape, activation="sigmoid"))

    return model

VGG-19

In [ ]:
def vgg_19(input_shape,output_shape):
    model = Sequential()

    model.add(Conv2D(input_shape=input_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    # Actual one is softmax
    model.add(Dense(output_shape, activation="sigmoid"))

    return model

model = vgg_16((500,150,3),1)

**ResNet-Network**

resnet-50

In [ ]:
def resnet_50(input_shape, output_shape):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def identity_block(tensor, f):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([x, tensor])
    output = ReLU()(x)
    return output
  
  
  def conv_block(tensor, f, s):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3, s)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(4*f, 1, strides=s)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    output = ReLU()(x)
    return output
  
  
  def resnet_block(x, f, r, s=2):
    x = conv_block(x, f, s)
    for _ in range(r-1):
      x = identity_block(x, f)
    return x
    
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = resnet_block(x, 64, 3, 1)
  x = resnet_block(x, 128, 4)
  x = resnet_block(x, 256, 6)
  x = resnet_block(x, 512, 3)
  
  x = GlobalAvgPool2D()(x)
  
  output = Dense(output_shape, activation='softmax')(x)
  
  model = Model(input, output)
  return model

ResNet-101

In [ ]:
def resnet_101(input_shape, output_shape):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def identity_block(tensor, f):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([x, tensor])
    output = ReLU()(x)
    return output
  
  
  def conv_block(tensor, f, s):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3, s)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(4*f, 1, strides=s)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    output = ReLU()(x)
    return output
  
  
  def resnet_block(x, f, r, s=2):
    x = conv_block(x, f, s)
    for _ in range(r-1):
      x = identity_block(x, f)
    return x
    
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = resnet_block(x, 64, 3, 1)
  x = resnet_block(x, 128, 4)
  x = resnet_block(x, 256, 23)
  x = resnet_block(x, 512, 3)
  
  x = GlobalAvgPool2D()(x)
  
  output = Dense(output_shape, activation='softmax')(x)
  
  model = Model(input, output)
  return model

ResNet-152

In [ ]:
def resnet_152(input_shape, output_shape):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def identity_block(tensor, f):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([x, tensor])
    output = ReLU()(x)
    return output
  
  
  def conv_block(tensor, f, s):
    x = conv_bn_rl(tensor, f)
    x = conv_bn_rl(x, f, 3, s)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)
    
    shortcut = Conv2D(4*f, 1, strides=s)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    output = ReLU()(x)
    return output
  
  
  def resnet_block(x, f, r, s=2):
    x = conv_block(x, f, s)
    for _ in range(r-1):
      x = identity_block(x, f)
    return x
    
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = resnet_block(x, 64, 3, 1)
  x = resnet_block(x, 128, 8)
  x = resnet_block(x, 256, 36)
  x = resnet_block(x, 512, 3)
  
  x = GlobalAvgPool2D()(x)
  
  output = Dense(output_shape, activation='softmax')(x)
  
  model = Model(input, output)
  return model

input_shape = 500, 150, 3

output_shape = 1

K.clear_session()

model = resnet_50(input_shape, output_shape)

**Sqeeze_and_Excitation_Network**

In [ ]:
def squeezenet(input_shape, output_shape):
  
  def fire(x, fs, fe):
    s = Conv2D(fs, 1, activation='relu')(x)
    e1 = Conv2D(fe, 1, activation='relu')(s)
    e3 = Conv2D(fe, 3, padding='same', activation='relu')(s)
    output = Concatenate()([e1, e3])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(96, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = fire(x, 16, 64)
  x = fire(x, 16, 64)
  x = fire(x, 32, 128)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = fire(x, 32, 128)
  x = fire(x, 48, 192)
  x = fire(x, 48, 192)
  x = fire(x, 64, 256)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = fire(x, 64, 256)
  x = Conv2D(output_shape, 1)(x)
  x = GlobalAvgPool2D()(x)
  
  output = Activation('softmax')(x)
  
  model = Model(input, output)
  return model

input_shape = 500, 150, 3

output_shape = 1

K.clear_session()

model = squeezenet(input_shape, output_shape)

**AlexNet**

In [ ]:
def alexnet(input_shape, output_shape):
  input = Input(input_shape)
  
  # actually batch normalization didn't exist back then
  # they used LRN (Local Response Normalization) for regularization
  x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(256, 5, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
  x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(output_shape, activation='softmax')(x)
  
  model = Model(input, output)
  return model

input_shape = 500, 150, 3

output_shape = 1

K.clear_session()

model = alexnet(input_shape, output_shape)

**Model Verification**

In [ ]:
input_shape = 500, 150, 3
output_shape = 1

K.clear_session()
model = alexnet(input_shape, output_shape)

In [ ]:
# Number of trainable parameters
import numpy as np
import tensorflow.keras.backend as K
np.sum([K.count_params(p) for p in model.trainable_weights])

64578113

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss="binary_crossentropy", optimizer= RMSprop(learning_rate=.001), metrics = ['accuracy'])

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import random
import numpy as np

DATASET_X1 = '/content/drive/MyDrive/Deep Learning/SugarCane_Quality/data/HoCP09-804'
DATASET_X2 = '/content/drive/MyDrive/Deep Learning/SugarCane_Quality/data/HoCP96-540'
DATASET_X3 = '/content/drive/MyDrive/Deep Learning/SugarCane_Quality/data/L01-299'
CATAGORIES = ["damaged","good"]

In [ ]:
#  For HoCP09-804 variety
dataset_X1 = []

# For HoCP96-540 variety
dataset_X2 = []

# For L01-299 variety
dataset_X3 = []

#  Defining image size
IMG_SIZE_X = 500
IMG_SIZE_Y = 150

def create_data(DATASET,training_data):
  for category in CATAGORIES:
    # Path to the image
    path = os.path.join(DATASET,category) 
    # Category Index
    class_num = CATAGORIES.index(category)
    for img in os.listdir(path):
      # Check for broken image
      try:
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array , (IMG_SIZE_X,IMG_SIZE_Y))
        training_data.append([new_array,class_num])
      except Exception as e:
        pass

create_data(DATASET_X1,dataset_X1)
create_data(DATASET_X2,dataset_X2)
create_data(DATASET_X3,dataset_X3)

print(len(dataset_X1))
print(len(dataset_X2))
print(len(dataset_X3))

1397
1072
405


In [ ]:
random.shuffle(dataset_X1)
random.shuffle(dataset_X2)
random.shuffle(dataset_X3)

train_dataset_X = []
train_dataset_y = []

test_dataset_X = []
test_dataset_y = []

def split_data(X):
  l = len(X)
  i = 0
  while i<((1*l)/10.0):
    train_dataset_X.append(X[i][0])
    train_dataset_y.append(X[i][1])
    i+=1
  while i<((1.5*l)/10.0):
    test_dataset_X.append(X[i][0])
    test_dataset_y.append(X[i][1])
    i+=1

split_data(dataset_X1)
split_data(dataset_X2)
split_data(dataset_X3)

print(len(train_dataset_X))
print(len(test_dataset_X))
print(len(train_dataset_y))
print(len(test_dataset_y))

289
143
289
143


In [ ]:
train_dataset_X = np.array(train_dataset_X).reshape(-1,IMG_SIZE_X,IMG_SIZE_Y,3)
train_dataset_y = np.array(train_dataset_y)

test_dataset_X = np.array(test_dataset_X).reshape(-1,IMG_SIZE_X,IMG_SIZE_Y,3)
test_dataset_y = np.array(test_dataset_y)

In [ ]:
print(len(train_dataset_X))
print(len(test_dataset_X))
print(len(train_dataset_y))
print(len(test_dataset_y))

289
143
289
143


In [ ]:
# Normalizing the data
train_dataset_X = train_dataset_X/255.0
test_dataset_X = test_dataset_X/255.0

In [ ]:
print(len(train_dataset_X))
print(len(test_dataset_X))
print(len(train_dataset_y))
print(len(test_dataset_y))

289
143
289
143


In [ ]:
model.fit(train_dataset_X,train_dataset_y,batch_size=32,epochs=20,validation_split=0.2)

Epoch 1/20
8/8 [==============================] - 8s 497ms/step - loss: 175.3586 - accuracy: 0.3636 - val_loss: 15.8915 - val_accuracy: 0.4310
Epoch 2/20
8/8 [==============================] - 1s 163ms/step - loss: 3.7521 - accuracy: 0.3636 - val_loss: 3.8082 - val_accuracy: 0.4310
Epoch 3/20
8/8 [==============================] - 1s 157ms/step - loss: 2.0722 - accuracy: 0.3636 - val_loss: 1.2477 - val_accuracy: 0.4310
Epoch 4/20
8/8 [==============================] - 1s 157ms/step - loss: 1.1666 - accuracy: 0.3636 - val_loss: 2.0126 - val_accuracy: 0.4310
Epoch 5/20
8/8 [==============================] - 1s 159ms/step - loss: 0.9052 - accuracy: 0.3636 - val_loss: 1.7268 - val_accuracy: 0.4310
Epoch 6/20
8/8 [==============================] - 1s 156ms/step - loss: 1.4592 - accuracy: 0.3636 - val_loss: 1.4449 - val_accuracy: 0.4310
Epoch 7/20
8/8 [==============================] - 1s 159ms/step - loss: 0.9467 - accuracy: 0.3636 - val_loss: 1.3857 - val_accuracy: 0.4310
Epoch 8/20
8/8 [=

In [ ]:
model.evaluate(test_dataset_X,test_dataset_y)

5/5 [==============================] - 1s 279ms/step - loss: 0.6767 - accuracy: 0.3287


[0.6766554117202759, 0.32867133617401123]